# Capítulo 12: Distribuindo TensorFlow em Dispositivos e Servidores

O Capítulo 12 do livro "Hands-On Machine Learning with Scikit-Learn & TensorFlow" foca em como utilizar o TensorFlow para **distribuir computações em múltiplos dispositivos (CPUs e GPUs) e executá-los em paralelo**. O objetivo principal é **acelerar o treinamento de grandes redes neurais** que, de outra forma, levariam dias ou semanas em uma única máquina, permitindo experimentação mais fácil e retreinamento frequente de modelos com novos dados.

## Principais Pontos Abordados:

### 1. Distribuição em uma Única Máquina
*   **Suporte a GPU**: O capítulo começa abordando o uso de GPUs para aceleração, mencionando que para suporte a GPU, é necessário instalar `tensorflow-gpu`. Ele também discute como verificar o uso da memória da GPU com `nvidia-smi` e como configurar o TensorFlow para alocar apenas parte da memória da GPU.
*   **Posicionamento de Dispositivos**: Explica como **fixar (pinning) operações em dispositivos específicos** (como "/cpu:0" ou "/gpu:0") usando `tf.device()`. O sistema de posicionamento dinâmico do TensorFlow (dynamic placer), que aloca operações automaticamente, é mencionado como algo que não foi liberado no código-fonte aberto, com usuários geralmente especificando regras de posicionamento para eficiência.
*   **Dependências de Controle**: Descreve as dependências de controle como uma forma de **adiar a avaliação de uma operação** até que todas as operações das quais ela depende tenham sido executadas, útil para gerenciar o uso de memória e a largura de banda de comunicação.

### 2. Múltiplos Dispositivos em Múltiplos Servidores (Cluster TensorFlow)
*   **Especificação de Cluster**: Introduz a ideia de um **cluster TensorFlow**, que consiste em **jobs** (trabalhos), como `"ps"` para **servidores de parâmetros** e `"worker"` para computações, e **tasks** (tarefas) que são os servidores TensorFlow individuais.
*   **Fragmentação de Variáveis (Sharding)**: Para modelos grandes, é útil **fragmentar os parâmetros em múltiplos servidores de parâmetros** para evitar a saturação de um único servidor. A função `replica_device_setter()` é apresentada para distribuir variáveis de forma balanceada (round-robin) entre as tarefas de "ps".
*   **Compartilhamento de Estado entre Sessões**: Em um ambiente distribuído, o estado das variáveis é gerenciado por **contêineres de recursos no próprio cluster**, não pelas sessões, permitindo que múltiplas sessões de cliente compartilhem as mesmas variáveis. `tf.container()` pode ser usado para criar escopos de variáveis únicos para evitar colisões de nomes.
*   **Comunicação Assíncrona com Filas**: O TensorFlow oferece **filas** (`tf.FIFOQueue`, `tf.RandomShuffleQueue`) para comunicação assíncrona entre diferentes partes do grafo ou entre clientes e o cluster. Isso é particularmente útil para entradas de comprimento variável, como sequências de palavras.
*   **Carregamento de Dados Direto do Grafo**: Para maior eficiência, é preferível **carregar os dados de treinamento diretamente do sistema de arquivos para o grafo do TensorFlow**. Isso evita a transferência de dados múltiplas vezes através do cliente. O capítulo introduz **operações de leitura (reader operations)** como `tf.decode_csv()` e funções de conveniência como `string_input_producer()`, juntamente com `QueueRunner` e `Coordinator`, para leitura eficiente de dados, inclusive de múltiplos arquivos por múltiplos threads.

### 3. Paralelização de Redes Neurais em um Cluster TensorFlow
*   **Múltiplas Redes Neurais**: Aborda como paralelizar o treinamento de múltiplas redes neurais:
    *   **Replicação "in-graph"**: Onde várias réplicas da rede neural são definidas dentro de um único grafo.
    *   **Replicação "between-graph"**: Onde cada rede neural tem seu próprio grafo separado e a sincronização é gerenciada por filas.
    *   Discussão sobre como lidar com **timeouts** em operações.
*   **Paralelismo de Modelo (Model Parallelism)**: Envolve **dividir um único modelo de rede neural em diferentes partes e executar cada parte em um dispositivo diferente**. Embora desafiador para redes totalmente conectadas devido à comunicação entre dispositivos, pode ser mais eficaz para arquiteturas específicas, como **redes neurais convolucionais (CNNs)** e **recorrentes (RNNs)**. Para RNNs profundas, pode-se fixar cada camada em uma GPU diferente.
*   **Paralelismo de Dados (Data Parallelism)**: Esta é uma abordagem comum onde a **rede neural é replicada em cada dispositivo**. Cada réplica executa uma etapa de treinamento simultaneamente usando um **mini-lote diferente**, e então os gradientes são agregados para atualizar os parâmetros do modelo.
    *   **Atualizações Síncronas**: Todas as réplicas esperam umas pelas outras para que os gradientes sejam agregados antes de atualizar os parâmetros.
    *   **Atualizações Assíncronas**: As réplicas atualizam os parâmetros independentemente, sem esperar pelas outras. Isso pode levar a mais etapas de treinamento por minuto, mas introduz o problema de **"gradientes obsoletos" (stale gradients)**.
    *   O capítulo discute os efeitos dos gradientes obsoletos e formas de mitigá-los (como reduzir a taxa de aprendizado, descartar ou escalar gradientes obsoletos, ajustar o tamanho do mini-lote e usar uma fase de aquecimento). Pesquisas do Google Brain sugeriram que o paralelismo de dados com atualizações síncronas usando algumas réplicas sobressalentes era o mais eficiente.
*   **Quantização**: Brevemente mencionada como uma técnica para reduzir o tamanho do modelo e acelerar as computações, especialmente para implantação em dispositivos móveis, ao diminuir a precisão (ex: de 16 para 8 bits) após o treinamento.

O capítulo conclui que, com essas ferramentas, é possível treinar redes neurais profundas em dúzias de servidores e GPUs, tornando o processo muito mais rápido.

## Implementação

## Exercícios

1. O que provavelmente está acontecendo se você receber um "CUDA_ERROR_OUT_OF_MEMORY" ao iniciar seu Tensorflow? O que pode ser feito em relação a isso?

Se você receber um erro CUDA_ERROR_OUT_OF_MEMORY ao iniciar seu programa TensorFlow, o que provavelmente está acontecendo é que o TensorFlow, por padrão, aloca toda a RAM disponível em todas as GPUs que lhe são visíveis na primeira vez que você executa um grafo. Isso significa que, se outros processos já estiverem em execução e tiverem alocado memória em pelo menos um dispositivo GPU visível, você não conseguirá iniciar um segundo programa TensorFlow enquanto o primeiro ainda estiver ativo.
Soluções:

A. Parar outros processos: A solução mais simples é interromper os outros processos que estão utilizando a memória da GPU e tentar novamente.

B. Dedicar GPUs diferentes a cada processo: Uma opção é usar a variável de ambiente CUDA_VISIBLE_DEVICES para que cada processo visualize e utilize apenas as placas GPU apropriadas. Por exemplo, você poderia iniciar dois programas da seguinte forma:
    ◦ $ CUDA_VISIBLE_DEVICES=0,1 python3 programa_1.py
    ◦ $ CUDA_VISIBLE_DEVICES=2,3 python3 programa_2.py O programa #1 verá apenas as GPUs 0 e 1, e o programa #2 verá apenas as GPUs 2 e 3.

C. Configurar o TensorFlow para alocar apenas uma fração da memória da GPU: Em vez de alocar toda a memória, você pode dizer ao TensorFlow para pegar apenas uma parte da memória da GPU. Isso é feito criando um objeto tf.ConfigProto, definindo sua opção gpu_options.per_process_gpu_memory_fraction para a proporção da memória total que ele deve alocar (por exemplo, 0.4 para 40%), e usando esta configuração ao criar a sessão. Por exemplo:

D. Dessa forma, dois programas podem ser executados em paralelo usando as mesmas placas GPU, desde que a soma das frações não exceda 1.

E. Permitir o crescimento da memória sob demanda: Você também pode instruir o TensorFlow a alocar memória apenas quando precisar, definindo config.gpu_options.allow_growth como True. No entanto, esta opção geralmente não é recomendada, pois o TensorFlow nunca libera a memória depois de alocá-la (para evitar fragmentação da memória), o que significa que você ainda pode ficar sem memória depois de um tempo. Além disso, pode ser mais difícil garantir um comportamento determinístico com esta opção, sendo geralmente preferível as opções anteriores.

2. Qual é a diferença entre fixar uma operação em um dispositivo e posicionar uma operação em um dispositivo?

Fixar uma operação em um dispositivo é a **sua instrução explícita** para o TensorFlow sobre onde você gostaria que uma operação fosse executada, geralmente feita usando `tf.device()`. Por outro lado, o **posicionamento** é a **decisão final do TensorFlow** sobre qual dispositivo realmente executará a operação, após considerar sua solicitação e outras restrições. Essa decisão final pode diferir da sua instrução de fixação se a operação não tiver uma implementação (kernel) para o dispositivo especificado, levando a uma exceção por padrão ou a um fallback para a CPU se o posicionamento suave (`allow_soft_placement`) estiver ativado, ou se houver requisitos de colocação, como uma operação que modifica uma variável precisar estar no mesmo dispositivo que a variável.

3. Se você estiver executando uma instalação do TensorFlow habilitada para GPU e utilizar apenas o posicionamento padrão, todas as operações serão posicionadas na primeira GPU?

Não, se você estiver executando uma instalação do TensorFlow habilitada para GPU e usar apenas o posicionamento padrão, **nem todas as operações serão posicionadas na primeira GPU**. Por padrão, o TensorFlow usará o "simple placer" (posicionador simples) que designa a GPU #0 como padrão para operações não fixadas. Contudo, isso só ocorrerá se todas as operações possuírem uma implementação (kernel) para GPU. Se uma ou mais operações não tiverem um kernel de GPU, o TensorFlow **levantará uma exceção por padrão**; se o "soft placement" estiver configurado como `True`, essas operações **serão automaticamente posicionadas na CPU**.

4. Se você ficar uma variável na "/gpu:0", ela pode ser utilizada por operações posicionadas na /gpu:1? Ou por operações posicionadas na "/cpu:0"? Ou por operações fixadas em dispositivos localizados em outros servidores? 

Sim, se você fixar uma variável na "/gpu:0", ela **pode ser utilizada por operações posicionadas na "/gpu:1", por operações posicionadas na "/cpu:0" e também por operações fixadas em dispositivos localizados em outros servidores**, desde que façam parte do mesmo cluster. O TensorFlow se encarrega automaticamente de adicionar as operações apropriadas para **transferir o valor da variável entre os dispositivos** conforme necessário.

5. Duas operações posicionadas no mesmo dispositivo podem rodar em paralelo?

Sim, duas operações posicionadas no mesmo dispositivo **podem rodar em paralelo** no TensorFlow, desde que não haja dependência de uma na saída da outra. O TensorFlow gerencia automaticamente a execução de operações em paralelo, utilizando diferentes *CPU cores* ou *GPU threads* no mesmo dispositivo.

6. O que é uma dependência de controle e quando você utiliza?

Uma dependência de controle é uma instrução que você dá ao TensorFlow para **adiar a avaliação de uma operação** até que outras operações específicas tenham sido executadas, mesmo que a operação adiada não dependa diretamente da saída das outras. Você a utiliza principalmente para otimizar o uso de recursos e evitar gargalos, como **postegar a avaliação de operações que consomem muita memória** até o último momento para liberar RAM para outras operações, ou para **executar operações intensivas em I/O sequencialmente** para não saturar a largura de banda de comunicação de um dispositivo.

7. Suponha que você treine uma DNN por dias em um cluster do Tensorflow e, imediatamente após o término do seu programa de treinamento, percebe que esqueceu de salvar o modelo ao utilizar um Saver. Seu modelo treinado foi perdido?

O modelo não será perdido. Em um cluster TensorFlow distribuído, os valores das variáveis do modelo são armazenados em **contêineres gerenciados pelo próprio cluster**, e não nas sessões individuais do cliente. Isso significa que, mesmo que seu programa cliente termine e a sessão seja fechada, os parâmetros do modelo permanecem ativos no cluster . Você precisaria abrir uma nova sessão para o cluster e usar um `Saver` para salvar o modelo, tomando cuidado para não inicializar as variáveis ou restaurar um modelo anterior, o que sobrescreveria seu modelo treinado .